In [2]:
import pandas as pd
import re
# from nltk.corpus import words
from nltk.tokenize import word_tokenize
import nltk
import numpy as np
#nltk.download('punkt')
from textblob import Word
pd.options.display.width = 0

import spacy
nlp = spacy.load('en_core_web_sm')
all_stopwords = nlp.Defaults.stop_words

pd.set_option('display.max_rows', 6000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 1000)
#pd.describe_option('display')
# import enchant

In [3]:
def find_in_english_dict(text):
    for w in word_tokenize(text):
        if Word(w.strip().lower()).spellcheck()[0][1]!=1.0 and len(re.findall(r"[A-Z]{2,}", w))>0:
          # print(w)
          return True
    return False

find_in_english_dict('HYPERTHROIDSM')
Word('allergy'.strip().lower()).spellcheck()

[('clergy', 0.48717948717948717),
 ('gallery', 0.28205128205128205),
 ('alley', 0.1282051282051282),
 ('allegro', 0.05128205128205128),
 ('allegory', 0.02564102564102564),
 ('allege', 0.02564102564102564)]

In [70]:
l = {'text_id':['A1','A2','A3','A4'], 'text':['a','b','c', 'd'], 'cui_l':[[1,2,5],[2,5], [],[]],
    'cui_g':[[1,2,3],[2,3],[4],[]], 'cui_d':[[1,2,4],[3], [], []]}
df = pd.DataFrame(data=l)
df['cui_len']=df['cui_l'].apply(lambda x: len(x))
df['cui_g_l']=(df['cui_g']+df['cui_d']).apply(lambda x: list(set(x)))
df['ID'] = ['ID'+ str(x) for x in df.index.tolist()]
df['cui_final'] = np.where(df['cui_len']!=0, df['cui_l'], df['cui_g_l'])
df_id_cui = df[['ID','cui_final']]
df, df_id_cui.explode('cui_final', ignore_index=True), 

(  text_id text      cui_l      cui_g      cui_d  cui_len       cui_g_l   ID  \
 0      A1    a  [1, 2, 5]  [1, 2, 3]  [1, 2, 4]        3  [1, 2, 3, 4]  ID0   
 1      A2    b     [2, 5]     [2, 3]        [3]        2        [2, 3]  ID1   
 2      A3    c         []        [4]         []        0           [4]  ID2   
 3      A4    d         []         []         []        0            []  ID3   
 
    cui_final  
 0  [1, 2, 5]  
 1     [2, 5]  
 2        [4]  
 3         []  ,
     ID cui_final
 0  ID0         1
 1  ID0         2
 2  ID0         5
 3  ID1         2
 4  ID1         5
 5  ID2         4
 6  ID3       NaN)